# The Danish Tax Burden

Imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')

# Data

In this paper we analyze and try to explain the development in the Danish tax burden in the years around the financial crisis, 2004-2016. To do so, we import and prepare a dataset containing information about the relevant variables from Statistikbanken; a process we will go through in this section. From this, we will create the variables necessary for the investigation and, finally, combine all data into one dataset pertaining specifically to this paper. 

## GDP

We start out by importing data from the NAN1 table from Statistikbanken. This table contains information about components of the danish national accounts, including the danish GDP. 

In [2]:
nan1_var = dst.get_variables(table_id='NAN1')
nan1_var

,id,text,elimination,time,values
0,TRANSAKT,transaction,False,False,"[{'id': 'B1GQK', 'text': 'B.1*g Gross domestic..."
1,PRISENHED,price unit,False,False,"[{'id': 'V_M', 'text': 'Current prices, (bill...."
2,Tid,time,False,True,"[{'id': '1966', 'text': '1966'}, {'id': '1967'..."


On this, we use the following (now muted) piece of code to identify all relevant variables and their values for selection:

In [3]:
#for id in ['TRANSAKT','PRISENHED','Tid']:
#    print(id)
#    values = nan1_var.loc[nan1_var.id == id,['values']].values[0,0]
#    for value in values:      
#        print(f' id = {value["id"]}, text = {value["text"]}')

Using this information, we select gross domestic product, GDP, (in both current and real prices) along with its real growth over time. 

In [4]:
time = ['2004','2005','2006','2007','2008','2009','2010','2011','2012', '2013', '2014', '2015', '2016']
variables_gdp = {'PRISENHED':['V_M', 'LAN_M', 'L_V'],'Tid': time}
gdp_api = dst.get_data(table_id = 'NAN1', variables=variables_gdp)
gdp_api.head(5)

,PRISENHED,TID,TRANSAKT,INDHOLD
0,"2010-prices, chained values, (bill. DKK.)",2004,B.1*g Gross domestic product,1751.0
1,Period-to-period real growth (per cent),2004,B.1*g Gross domestic product,2.7
2,"Current prices, (bill. DKK.)",2004,B.1*g Gross domestic product,1506.0
3,"2010-prices, chained values, (bill. DKK.)",2005,B.1*g Gross domestic product,1792.0
4,Period-to-period real growth (per cent),2005,B.1*g Gross domestic product,2.3


For simplicity (and to lower the risk of bugs), we rename all columns, units and variables:

In [5]:
# a. code for creating dictonaries containing new names of columns, units and variables. 
columns_dict = {}
columns_dict['TRANSAKT'] = 'variable'
columns_dict['PRISENHED'] = 'unit'
columns_dict['TID'] = 'year'
columns_dict['INDHOLD'] = 'value'

var_dict = {} # var is for variable
var_dict['B.1*g Gross domestic product'] = 'gdp'

unit_dict = {}
unit_dict['2010-prices, chained values, (bill. DKK.)'] = 'gdp_real'
unit_dict['Current prices, (bill. DKK.)'] = 'gdp_nominal'
unit_dict['Period-to-period real growth (per cent) '] = 'gdp_growth'

# b. renaming
gdp_api.rename(columns = columns_dict, inplace=True)

for key,value in var_dict.items():
    gdp_api.variable.replace(key,value,inplace=True)

for key,value in unit_dict.items():
    gdp_api.unit.replace(key,value,inplace=True)

gdp_api.head(5)

,unit,year,variable,value
0,gdp_real,2004,gdp,1751.0
1,gdp_growth,2004,gdp,2.7
2,gdp_nominal,2004,gdp,1506.0
3,gdp_real,2005,gdp,1792.0
4,gdp_growth,2005,gdp,2.3


To achieve a better overview (and to simultaniously get rid of the 'TRANSAKT' column), we pivot the table.

In [6]:
gdp_api = gdp_api.pivot_table('value', 'year', 'unit')
gdp_api.head(10)

unit,gdp_growth,gdp_nominal,gdp_real
year,,,
2004,2.7,1506.0,1751.0
2005,2.3,1586.0,1792.0
2006,3.9,1682.3,1862.1
2007,0.9,1738.8,1879.0
2008,-0.5,1801.5,1869.4
2009,-4.9,1722.1,1777.7
2010,1.9,1810.9,1810.9
2011,1.3,1846.9,1835.1
2012,0.2,1895.0,1839.3


## Taxes

The second step is now to create a table containing general information about tax revenues. This kind of information is contained in the "SKAT"-table from Statistikbanken, which we import and transform, as with GDP.

In [7]:
tax_var = dst.get_variables(table_id='SKAT')
tax_var

,id,text,elimination,time,values
0,TYPE,type,True,False,"[{'id': 'TOTAL', 'text': 'Taxes and duties, to..."
1,Tid,time,False,True,"[{'id': '1947', 'text': '1947'}, {'id': '1948'..."


The relevant variables are again identified (muted) and we import the relevant data:

In [8]:
# a. identifying variables
#for id in ['TYPE','Tid']:
#    print(id)
#    values = skat_var.loc[skat_var.id == id,['values']].values[0,0]
#    for value in values:      
#        print(f' id = {value["id"]}, text = {value["text"]}')

# b. importing data
variables_skat = {'TYPE':['TOTAL', '1', '1.2','1.3', '4.3'],'Tid': time}
tax_api = dst.get_data(table_id = 'SKAT', variables=variables_skat)
tax_api.head(6)

,TYPE,TID,INDHOLD
0,"1. Income taxes, total",2004,429918714
1,"1.2. Corporation tax, etc.",2004,43745483
2,1.3. Tax on yields of certain pension scheme a...,2004,23629731
3,4.3. Motor vehicle duties,2004,8861243
4,"Taxes and duties, total",2004,700893135
5,"1. Income taxes, total",2005,473503284


In [9]:
# a. new names
columns_dict_tax = {}
columns_dict_tax['TYPE'] = 'tax'
columns_dict_tax['TID'] = 'year'
columns_dict_tax['INDHOLD'] = 'value'

var_dict_tax = {}
var_dict_tax['Taxes and duties, total'] = 'tax_total'
var_dict_tax['1.2. Corporation tax, etc.'] = 'tax_corp'
var_dict_tax['1. Income taxes, total'] = 'tax_income'
var_dict_tax['1.3. Tax on yields of certain pension scheme assets'] = 'tax_pension'
var_dict_tax['4.3. Motor vehicle duties'] = 'tax_vehic'

# b. renaming
tax_api.rename(columns = columns_dict_tax, inplace=True)

for key,value in var_dict_tax.items():
    tax_api.tax.replace(key,value,inplace=True)

# c. transformming table
tax_api = tax_api.pivot_table('value', 'year', 'tax')
tax_api.head(10)

tax,tax_corp,tax_income,tax_pension,tax_total,tax_vehic
year,,,,,
2004,43745483,429918714,23629731,700893135,8861243
2005,54596843,473503284,37243509,764015560,9287168
2006,62165644,477965264,12551607,784341163,9823659
2007,54870607,486844336,4561497,809988227,10229661
2008,46049479,491450126,9534927,808883678,10400869
2009,32886092,476878509,8784250,777374850,10464217
2010,41056740,504988660,36535130,818683609,10594290
2011,40286382,513117080,38047746,835833903,10457067
2012,49474123,540729484,43551938,870915696,10596369


## Merging

Finally, we merge out two datasets.

In [10]:
gdptax = pd.merge(gdp_api, tax_api, on='year', how='left')
gdptax.head(10)

,gdp_growth,gdp_nominal,gdp_real,tax_corp,tax_income,tax_pension,tax_total,tax_vehic
year,,,,,,,,
2004,2.7,1506.0,1751.0,43745483,429918714,23629731,700893135,8861243
2005,2.3,1586.0,1792.0,54596843,473503284,37243509,764015560,9287168
2006,3.9,1682.3,1862.1,62165644,477965264,12551607,784341163,9823659
2007,0.9,1738.8,1879.0,54870607,486844336,4561497,809988227,10229661
2008,-0.5,1801.5,1869.4,46049479,491450126,9534927,808883678,10400869
2009,-4.9,1722.1,1777.7,32886092,476878509,8784250,777374850,10464217
2010,1.9,1810.9,1810.9,41056740,504988660,36535130,818683609,10594290
2011,1.3,1846.9,1835.1,40286382,513117080,38047746,835833903,10457067
2012,0.2,1895.0,1839.3,49474123,540729484,43551938,870915696,10596369


# Analysis 

In our analysis, we start with the development in the danish real GDP. The gross domestic product isn't only necessary for calculating the tax burden, GDP growth is also integral to explaining its developments. Firstly, when times are good (i.e. during the expansions and peaks of business cycles) we tend to see increasing employment and higher levels of income. In a progressive tax system, this naturally entails an increase in the tax burden. Furthermore, people tend to shift their consumption towards luxury good, which are often more heavily taxed. Other factors are also relevant, which we will get into later. 


In [32]:
%matplotlib widget
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax2.bar(time, height = gdp_api['gdp_growth'], width = 0.8)
ax1.plot(time, gdp_api['gdp_real'], 'g')
ax1.grid(False)

ax1.set_xlabel('Year')
ax1.set_ylabel('Bill. DKK', color='g')
ax2.set_ylabel('Pct.', color='b')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In the figure above, we see the twelve year development in Danish real GDP. Here we note the business cycle expansion and peak from 2004-2007, after which the economy dives into a recession in 2008-2009, due to the financial crisis. From there the economy begins its recovery with relatively low growth rates from 2010-2014, until it enters another expansion in the following years. 

To calculate the tax burden, we use the formula
$$\text{Tax burden} = \frac{\text{Total tax revenue}}{\text{GDP in market prices}}$$
and set up our data in numpy arrays:

In [12]:
# a. creating arrays
#  i. nominal gdp
gdp_nom = np.array(gdptax['gdp_nominal'])
#  ii. tax revenue
tax_tot = np.array(gdptax['tax_total']) 

# b. calculating taxburden
taxburden = 100*(tax_tot/1000000)/gdp_nom

# c. descriptives
df_taxburden=pd.DataFrame(taxburden)
df_taxburden.head(5)
df_taxburden.describe()

,0
count,13.000000
mean,46.521395
std,1.379137
min,44.900565
25%,45.256045
50%,46.540049
75%,47.068949
max,49.815039


The average tax burden is 46,5 pct. Doing the period from 2004 to 2016 the lowest tax burden was 44,9 pct. and the highest tax burden was 49,8 pct. 

We're are now able to also illustrate the development in the tax burden.

In [13]:
fig = plt.figure()
plt.bar(time, taxburden, color = 'y')
plt.xlabel('Year')
plt.ylabel('Pct.')
plt.ylim(42, 51)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(42.0, 51.0)

From this figure, we notice the relatively stable level of tax burden around the average of 46.5 pct., during the aforementioned expansion and peak periods, with the exception of an usually high level in 2005. At the onset of the recession the level takes a visible dive to around 45pct., after which it remains pretty much stable for the period 2008-2011. In 2012 it slowly recovers and suddenly takes a jump upwards in 2013 to around 47pct., from which it booms to its highest level of 49,8pct. in 2014. The following two years, 2015-2016, it seems to be making its way back to around its initial level.

## Causes

As explained in the previous section, other factors than just the business cycles seem to be important in explaining the movements in the tax burden. Of these factors, political decisions seem to be the most important. In *Figure 3* we have shown the development of the total tax burden compared to the development of other chosen taxes. From this it can be seen, that the tax burden related to income taxes seem to the main driver of the total tax burden. During the concerned period there have been different political actions, which have influenced the total tax burden through taxation of income.

The significant rise in taxes during the period 2012-2015, are in large part due to a restructuring of capital pensions. In this period, everyone with a capital pension was offered to pay their future pension taxes in the years 2013-2015 at a lower tax rate, instead of at the time of payment (as is usually done). This advanced tax payments of approximately 29 bill. DKK in 2013, 59 bill. DKK in 2014, and 18 bill. DKK in 2015 - which, all else equal, of course caused a rise in the tax burden in these three years. This increase was largely due to an increase in the taxburden related to income taxes, which is as expected.

In 2004 a specific pension scheme ('Den Særlige Pensionsopsparing', or simply 'SP') was suspended. This, among other things, resulted in a larger tax burden, since the SP was deducted from tax payments. This could in part explain the high level of tax burden in 2005.

In [14]:
tax_inc = np.array(gdptax['tax_income'])
taxb_inc = 100*(tax_inc/1000000)/gdp_nom
tax_corp = np.array(gdptax['tax_corp'])
taxb_corp = 100*(tax_corp/1000000)/gdp_nom
tax_pension = np.array(gdptax['tax_pension'])
taxb_pension = 100*(tax_pension/1000000)/gdp_nom
tax_vehic = np.array(gdptax['tax_vehic'])
taxb_vehic = 100*(tax_vehic/1000000)/gdp_nom

In [31]:
%matplotlib widget 
#A. Creating figure
fig = plt.figure(3, figsize=(8,3))

#A.1 First subplot
ax1 = fig.add_subplot(121)

#A.1.1 Formatting first subplot
ax1.set_xticks(range(13))
ax1.set_xticklabels(time, fontdict = {'fontsize': 8})
plt.xticks(rotation = 45)
ax1.set_ylim(40,52)

#A.1.2 Entering data into first subplot
ax1.bar(time, taxburden, label = "total taxburden", color = 'y', zorder = 1000)
ax1_2 = ax1.twinx() #create second y_axis
ax1_2.plot(time, taxb_inc, label = "taxburden from income taxes", color = 'b') #data plotted on second y-axis

#A.1.3 Final fomratting of first subplot
ax1_2.set_ylim(26,34)
ax1_2.grid(False) 
ax1.set_xlabel('Year')
ax1.set_ylabel('Pct.', color='y')
ax1_2.set_ylabel('Pct.', color='b')
ax1.legend()
ax1_2.legend(loc = 'upper right', bbox_to_anchor=(1.3,1.12))

#A.2 Second subplot
ax2 = fig.add_subplot(122)

#A.2.1 Formatting second subplot
ax2.set_xticks(range(13))
ax2.set_xticklabels(time, fontdict = {'fontsize': 8})
plt.xticks(rotation = 45)

#A.2.2 Entering data into second subplot
ax2.plot(time, taxb_corp, label = 'Corporate tax, etc.', color = 'y')
ax2.plot(time, taxb_pension, label = 'Pension tax', color = 'b')
ax2_2 = ax2.twinx() #create second y-axis
ax2_2.plot(time, taxb_vehic, label = 'Motor vehicle duties (r.h. axis)', color = 'r') #data plotted on second y-axis

#A.2.3 Final fomratting of second subplot
ax2_2.grid(False)
#ax2.set_ylim(40,52)
ax2.set_xlabel('Year')
ax2.set_ylabel('Pct.')
ax2_2.set_ylabel('Pct.', color = 'red')
ax2.legend()
ax2_2.legend(loc = 'upper right', bbox_to_anchor=(1.23,1.12))
plt.tight_layout(pad = 1) #create space between plots

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Append

Finally we want to store all of our data. We therefore create a final table, inlucing all the variables used in this notebook, and export it as a CSV file. 
First step is to convert our arrays to dataframes.

In [16]:
# a. converting arrays to dataframes
df_taxb_all = pd.DataFrame({'tot_taxb':taxburden, 'tot_corp':tax_corp, 'tot_inc':tax_inc, 'tot_vehic':tax_vehic, 'taxb_pens':taxb_pension}, index=gdptax.index)
df_taxb_all.head()

,tot_taxb,tot_corp,tot_inc,tot_vehic,taxb_pens
year,,,,,
2004,46.540049,43745483,429918714,8861243,1.569039
2005,48.172482,54596843,473503284,9287168,2.348267
2006,46.623145,62165644,477965264,9823659,0.746098
2007,46.583174,54870607,486844336,10229661,0.262336
2008,44.900565,46049479,491450126,10400869,0.529277


We now append out new DataFrame to the rest of our dataset

In [17]:
# b. joining DataFrames
TaxBurdenData = gdptax.join(df_taxb_all)
TaxBurdenData.head(12)

,gdp_growth,gdp_nominal,gdp_real,tax_corp,tax_income,tax_pension,tax_total,tax_vehic,tot_taxb,tot_corp,tot_inc,tot_vehic,taxb_pens
year,,,,,,,,,,,,,
2004,2.7,1506.0,1751.0,43745483,429918714,23629731,700893135,8861243,46.540049,43745483,429918714,8861243,1.569039
2005,2.3,1586.0,1792.0,54596843,473503284,37243509,764015560,9287168,48.172482,54596843,473503284,9287168,2.348267
2006,3.9,1682.3,1862.1,62165644,477965264,12551607,784341163,9823659,46.623145,62165644,477965264,9823659,0.746098
2007,0.9,1738.8,1879.0,54870607,486844336,4561497,809988227,10229661,46.583174,54870607,486844336,10229661,0.262336
2008,-0.5,1801.5,1869.4,46049479,491450126,9534927,808883678,10400869,44.900565,46049479,491450126,10400869,0.529277
2009,-4.9,1722.1,1777.7,32886092,476878509,8784250,777374850,10464217,45.141098,32886092,476878509,10464217,0.510089
2010,1.9,1810.9,1810.9,41056740,504988660,36535130,818683609,10594290,45.208659,41056740,504988660,10594290,2.017512
2011,1.3,1846.9,1835.1,40286382,513117080,38047746,835833903,10457067,45.256045,40286382,513117080,10457067,2.060087
2012,0.2,1895.0,1839.3,49474123,540729484,43551938,870915696,10596369,45.958612,49474123,540729484,10596369,2.298255


We now export our dataset as a CSV file

In [18]:
# c. exporting csv file
TaxBurdenData.to_csv('TaxBurdenData.csv', index=False)

# Conclusion

The danish tax burden was on average 45,6 pct. between 2004 and 2016. 

Business cycles seem to influence the tax burden positively, which for example can be seen by the dive in both GDP and the danish tax burden following the financial crisis in 2008. However, there seem to be years where the overall economic growth cannot explain the movements in the tax burden. 

These movements, which are not explained by the economic cycle, may be caused by political decisions. This is  the case after the restructuring of capital pensions, which caused the tax burden to increase significantly. 

Overall, the tax burden seem to give a picture of the danish tax system, but it is also important to keep in mind, that the tax burden also reflects components, which does not say very much about the general tax system, as for example when the tax burden increased in 2005, which was not caused by any changes in tax rates but because of suspension of a specific pension scheme ('Den Særlige Pensionsopsparing')